# Extract needle labelmaps

In [1]:
from __future__ import division
%matplotlib inline
%pylab inline

import pyprind
import glob
import os
import SimpleITK as sitk
import numpy as np
import nrrd
import numpy as np
from sklearn import datasets, svm, metrics, decomposition
from sklearn.externals import joblib
import time
from joblib import Parallel, delayed  
import multiprocessing
num_cores = multiprocessing.cpu_count()
USERPATH = os.path.expanduser("~")
print(USERPATH)
from mpl_toolkits.mplot3d import Axes3D
import six.moves.cPickle as pickle

Populating the interactive namespace from numpy and matplotlib
/Users/guillaume


## Functions to load the cases in slicer and extract positive or negative patches

In [5]:
# read file containing path of each MRML scenes
with open('/Users/guillaume/Dropbox/GYN Cases/scenes.txt') as f:
    lines = f.readlines()

paths = []
cases = []
for line in lines:
    paths.append('/Users/guillaume/Dropbox/GYN Cases' + line[1:-1])
    cases.append(int(line.lstrip('./Case  ')[1:3]))


def startExtractingRandomCubes(start=0):
    '''
    Extract Random Negative patches from all cases
    '''
    with open('/Users/guillaume/Dropbox/GYN Cases/scenes.txt') as f:
        lines = f.readlines()
    paths = []
    cases = []
    for line in lines:
        paths.append('/Users/guillaume/Dropbox/GYN Cases' + line[1:-1])
        cases.append(int(line.lstrip('./Case  ')[1:3]))
    for k, path in enumerate(paths[start:]):
        spath = path.split('/')
        path = "/".join(spath[:-1]) + '/*.nrrd'
        saveRandomCubes(path, cases[k], 30)

def saveRandomCubes(path, number, numberOfSamples=30):
    '''
    Extract random negative patches. 
    TODO: check if it isn't actually a positive one
    '''
    try:
        volume = nrrd.read(glob.glob(path)[0])[0]
        for i in range(numberOfSamples):
            xR = np.random.randint(10,40)
            yR = np.random.randint(10,40)
            zR = np.random.randint(10,40)

            # sImage = sitk.GetImageFromArray(im[30:30+40, 30:30+40, 30:30+40])
            sImage = sitk.GetImageFromArray(volume)
            #
            rimage = sitk.Image(20,20,20, sitk.sitkFloat32)
            rimage.SetSpacing([1,1,1])
            # rimage.SetDirection([-1,0,0,0,-1,0,0,0,1])
            # rimage.SetOrigin([xR,yR,zR]) #-np.array([1,1,1]
            sImage.SetOrigin([-xR,-yR,-zR])
            rimage.SetOrigin([10,10,10])
            #rimage.SetDirection([1,0,0]
            tx = sitk.Transform()
            #
            lilImage = sitk.Resample(sImage, rimage, tx, sitk.sitkNearestNeighbor, sitk.sitkFloat32)
            filename = "/Users/guillaume/Dropbox/2016-projectweekdata/RandomCubes20-1/%d-%d-random_%d.nrrd" % (number, np.random.randint(9999999), i)
            sitk.WriteImage( lilImage, filename )
    except:
        pass

In [6]:
# startExtracting(0)

In [4]:
# p = glob.glob('/Users/guillaume/Dropbox/GYN Cases/Case  007/NRRD/Manual_CY/*.nrrd')

## Functions to convert manual needle segmentation to LabelMaps in Slicer

In [ ]:
def labelMapFromNeedle(inputVolume, needleID, value, caseNumber):
    '''
    Convert a needle to a labelmap, save it and remove the node from the scene.
    '''
    outputLabelMap = slicer.vtkMRMLLabelMapVolumeNode()
    slicer.mrmlScene.AddNode(outputLabelMap)
    params = {'sampleDistance': 0.1, 'labelValue': value, 'InputVolume': inputVolume.GetID(),
              'surface': needleID, 'OutputVolume': outputLabelMap.GetID()}
    slicer.cli.run(slicer.modules.modeltolabelmap, None, params, wait_for_completion=True)
    # slicer.mrmlScene.GetNodeByID("vtkMRMLSliceNodeRed").SetUseLabelOutline(True)
    # slicer.mrmlScene.GetNodeByID("vtkMRMLSliceNodeYellow").SetUseLabelOutline(True)
    # slicer.mrmlScene.GetNodeByID("vtkMRMLSliceNodeRed").RotateToVolumePlane(outputLabelMap)
    # slicer.mrmlScene.GetNodeByID("vtkMRMLSliceNodeYellow").RotateToVolumePlane(outputLabelMap)
    slicer.util.saveNode(outputLabelMap, '/Users/guillaume/Projects/LabelMaps/%d/needle-%d.nrrd'%(caseNumber,value))
    # slicer.mrmlScene.RemoveAllObservers()
    slicer.mrmlScene.RemoveNodeReferences(outputLabelMap)
    slicer.mrmlScene.RemoveNode(outputLabelMap)
    return 0

def startExtractingNeedles(start=0):
    '''
    Save the needles as individual labelmaps. Read the file the scenes.txt to obtain paths.
    '''
    with open('/Users/guillaume/Dropbox/GYN Cases/scenes.txt') as f:
        lines = f.readlines()
    paths = []
    cases = []
    for line in lines:
        paths.append('/Users/guillaume/Dropbox/GYN Cases' + line[1:-1])
        cases.append(int(line.lstrip('./Case  ')[1:3]))
    for k, path in enumerate(paths[start:]):
        slicer.util.loadScene(path)
        nodes = slicer.util.getNodes("manual-seg*")
        backgroundNode = slicer.app.layoutManager().sliceWidget("Red").sliceLogic().GetBackgroundLayer().GetVolumeNode()
        slicer.util.saveNode(backgroundNode, '/Users/guillaume/Projects/LabelMaps/%d/case.nrrd'%(cases[k+start]))
        for i, node in enumerate(nodes.values()):
            labelMapFromNeedle(backgroundNode, node.GetID(), i+1, cases[k+start])
            slicer.mrmlScene.RemoveNodeReferences(node)
            # slicer.mrmlScene.RemoveObservers(node)
            slicer.mrmlScene.RemoveNode(node)
        print("start removing nodes------")
        # slicer.mrmlScene.RemoveAllObservers()
        slicer.mrmlScene.RemoveNodeReferences(backgroundNode)
        slicer.mrmlScene.RemoveNode(backgroundNode)
        # slicer.mrmlScene.Clear(0)

def extract(k):
    '''
    Extract needles of just one case.
    '''
    slicer.util.loadScene(paths[k])
    nodes = slicer.util.getNodes("manual-seg*")
    backgroundNode = slicer.app.layoutManager().sliceWidget("Red").sliceLogic().GetBackgroundLayer().GetVolumeNode()
    slicer.util.saveNode(backgroundNode, '/Users/guillaume/Projects/LabelMaps/%d/case.nrrd'%(cases[k]))
    for i, node in enumerate(nodes.values()):
        labelMapFromNeedle(backgroundNode, node.GetID(), i+1, cases[k])
        # slicer.mrmlScene.RemoveNode(node)
    print("start removing nodes------")
    slicer.mrmlScene.RemoveAllObservers()
    slicer.mrmlScene.RemoveNodeReferences(backgroundNode)
    slicer.mrmlScene.RemoveNode(backgroundNode)
    # slicer.mrmlScene.Clear(0)